In [1]:
!pip install --upgrade --quiet pip
!pip install --quiet farm-haystack[colab,faiss]==1.17.2
print('pip install haystack complete.')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
chex 0.1.82 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scip

In [2]:
URL = 'https://www.intelligence.senate.gov/sites/default/files/publications/CRPT-113srpt288.pdf'
WORKING = '/kaggle/working/'

import os
import requests

# https://stackoverflow.com/a/56951135
def download(url: str, dest_folder: str):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)  # create folder if it does not exist

    filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names
    file_path = os.path.join(dest_folder, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file=file_path, mode='wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))


short_name = os.path.basename(URL)
if not os.path.exists(WORKING + short_name):
    download(url=URL, dest_folder=WORKING)
    print('downloaded {} to {}'.format(URL, WORKING))
else:
    print('file {} already exists.'.format(WORKING + short_name))
pdf_paths=[WORKING + short_name]

saving to /kaggle/working/CRPT-113srpt288.pdf
downloaded https://www.intelligence.senate.gov/sites/default/files/publications/CRPT-113srpt288.pdf to /kaggle/working/


In [3]:
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str='Flat')
print('created our document store')

created our document store


In [4]:
!apt-get install --quiet poppler-utils -y
print('pip install poppler utils complete.')
!sudo apt-get update
!sudo apt-get install --quiet poppler-utils -y
print('pip install poppler complete for real and for true')

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 40 not upgraded.
Need to get 2562 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Ign:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
Ign:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.3
Err:2 http://security.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.3
  404  Not

In [5]:
# Initialize a PDFToTextConverter to convert the PDF to text
from haystack.utils import convert_files_to_docs, fetch_archive_from_http, clean_wiki_text
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PDFToTextConverter, PreProcessor, DensePassageRetriever
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
print('many imports complete.')

many imports complete.


In [6]:
#replace this paths with your input Data path

# Convert the PDF documents to Haystack-compatible format
pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=['en'])
pdf_docs = []

for pdf_path in pdf_paths:
    doc = pdf_converter.convert(file_path=pdf_path, meta=None)[0]
    pdf_docs.append(doc)
print('converted {} documents'.format(len(pdf_docs)))

pdftotext version 0.86.1
Copyright 2005-2020 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011 Glyph & Cog, LLC
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weight
Syntax Warning: Invalid Font Weigh

converted 1 documents


In [7]:
# Initializing the PreProcessor to clean and split the text
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)
processed_docs = preprocessor.process(pdf_docs)
print('document processing done.')

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

document processing done.


## Initializing the Retriever

We use a `DensePassageRetriever` and we invoke `update_embeddings` to index the embeddings of documents in the `FAISSDocumentStore`.

In [8]:
# Initialize a DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='vblagoje/dpr-question_encoder-single-lfqa-wiki',
    passage_embedding_model='vblagoje/dpr-ctx_encoder-single-lfqa-wiki',
)
print('built retriever.')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


built retriever.


In [9]:
document_store.write_documents(processed_docs)
document_store.update_embeddings(retriever)
print('updated embeddings.')

Writing Documents:   0%|          | 0/4841 [00:00<?, ?it/s]

Updating Embedding:   0%|          | 0/4804 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/4816 [00:00<?, ? Docs/s]

updated embeddings.


In [10]:
from haystack.nodes import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')
print('built generator.')

built generator.


In [11]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path='deepset/deberta-v3-large-squad2')
print('built the reader.')

built the reader.


In [12]:
from haystack.pipelines import GenerativeQAPipeline
from haystack.pipelines import ExtractiveQAPipeline

generative_pipeline = GenerativeQAPipeline(generator, retriever)
extractive_pipeline = ExtractiveQAPipeline(reader, retriever)

In [13]:
query = 'Who was shown photographs?'
result = extractive_pipeline.run(query=query, params={'Retriever': {'top_k': 5}, 'Reader': {'top_k': 1}})
print(result)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'Who was shown photographs?', 'no_ans_gap': 9.179399490356445, 'answers': [<Answer {'answer': 'detainees', 'type': 'extractive', 'score': 0.985458254814148, 'context': ' photo was Issa al-Hindi, the CIA\'s\n| requested the photo be "shown to detainees" and requested "immediate\nAccording to a CIA cable dated June 17, 200', 'offsets_in_document': [{'start': 98, 'end': 107}], 'offsets_in_context': [{'start': 71, 'end': 80}], 'document_ids': ['2f451b3d2447defa92be79e7d5939cf1'], 'meta': {'_split_id': 2128, 'vector_id': '617'}}>], 'documents': [<Document: {'content': 'officials, who responded that this identification was "extremely\nCIA detainee Khallad bin Attash was shown the same photograph and stated that\nthe photo "definitely" was not Issa.1520\nCIA officers wrote that KSM "is obstructing our ability\nto acquire good information" on Issa and noting that KSM has "misidentified photos when he\nknows we are fishing" and "misleads us on telephone numbers.', 'content_type': 'tex

In [14]:
print(result['answers'][0].answer)

detainees


In [15]:
piperun=generative_pipeline.run(
    query='Who were shown photographs?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure if this is what you're looking for, but I can tell you that there was a photo of a man named Abu Zubaydah that was shown to Khalid Sheikh Mohammed (aka Osama bin Laden). The photo was taken by the FBI, and the photo was sent to the CIA, who then showed it to the other detainees, who all identified the photo as that of al-Hindi.


In [16]:
piperun=generative_pipeline.run(
    query='Who was Abu Zubaydah and what happened to him?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Abu Zubaydah was a member of Al-Qaeda who was captured by the CIA in 2002. He was put in solitary confinement for the rest of his life.


In [17]:
piperun=generative_pipeline.run(
    query='When and where did Abu Zubaydah die?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

Abu Zubaydah was taken into CIA custody on March 2002, and was shortly thereafter hospitalized until April 15, 2002, when he was transferred to a detention facility in Afghanistan, where he spent the rest of his life.


In [18]:
piperun=generative_pipeline.run(
    query='Who else is mentioned in the report?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not an expert on the subject, but I can tell you that there are a number of people who are mentioned in the report. I'm not sure if they are mentioned by name, but there are several people who were involved in the CIA's torture program. The most famous of these people is Valerie Plame, who was the head of the Office of Human Resources for the CIA during the Bush administration. Plame was responsible for the torture program that was used by the CIA in the early 2000s. She was the first person to come to the attention of the Senate Intelligence Committee, and she was one of the first people to testify before the committee. She also testified before the Senate Select Committee on Intelligence (SCI), which is the committee that investigates allegations of government misconduct. She testified before both the House Intelligence Committee and the Senate Armed Services Committee, both of which are part of the Intelligence Committee.


In [19]:
piperun=generative_pipeline.run(
    query='Who else is mentioned in the report apart from Valerie Plame?', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I don't know much about Plame, but I do know a lot about the CIA's use of torture. The CIA's torture program was called "Enhanced Interrogation Techniques" (EITs) and they were used on a number of people. One of them was Abu Zubaydah, who was an American citizen who was captured by the CIA. He was tortured in a variety of ways, including waterboarding, gas masking, and other methods. The most famous of these techniques was waterboarding. The reason for this was that the CIA wanted to make sure that the people they were interrogating didn't know who they were talking to, and that they didn't have any reason to believe that they were being tortured. The EITs were used to try to get people to confess to things that they weren't supposed to be confessing to. For example, the CIA used waterboarding to get a confession from Khalid Sheikh Mohammed, who had been tortured by the


In [20]:
piperun=generative_pipeline.run(
    query='Please tell me everything about Khalid Sheikh Mohammed.', params={'Retriever': {'top_k': 5}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I don't know much about Khalid Sheikh Mohammed, but I can tell you a little bit about him. He was born in Saudi Arabia, and grew up in the United States. He went to school in the U.S. in the early 2000s, and graduated from high school in 2001. He then went to work for the CIA, where he worked as a translator, translator, and translator. He also worked for the FBI for a few years, before being transferred to Guantanamo Bay. During his time in Guantanamo, he was subjected to a variety of interrogation techniques, including, but not limited to, the use of waterboarding, which is a form of forced waterboarding. The waterboarding technique is used to force people to confess to crimes they didn't commit, such as murder, rape, and other crimes that they did not commit. This technique is known as "enhanced interrogation" (ELI5 for short), and it is used in conjunction with other techniques such as


In [21]:
piperun=generative_pipeline.run(
    query='Please give some examples of enhanced interrogation techniques.', params={'Retriever': {'top_k': 1}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure what you mean by "enhanced interrogation techniques", but I'll try to give you a few examples. The first one I can think of off the top of my head is waterboarding. Waterboarding is a technique used by the CIA to extract information from suspects. This information is then used to determine whether or not the suspect is a threat to the U.S. or other countries.


In [22]:
piperun=generative_pipeline.run(
    query='Please give some examples of enhanced interrogation techniques.', params={'Retriever': {'top_k': 2}} 
)
for answer in piperun['answers']:
    print(answer.answer)

I'm not sure what you mean by "enhanced interrogation techniques", but I'll try to give you a few examples. The first one I can think of off the top of my head is waterboarding. Waterboarding is a technique used by the CIA to extract information from suspects. The idea is that if you can get a suspect to confess to a crime, you can then interrogate them until they confess to the crime. If you can't get them to confess, you don't have to torture them any more. The second one I think you're referring to is the waterboarding of Khalid Sheikh Mohammed. Khalid was an American citizen who was captured by the Saudi Arabian government. He was taken to a prison in Saudi Arabia, where he was kept in a waterboarding room. The waterboarding was used to get information from him. The reason they used waterboarding is because it was the most effective method of extracting information from a suspect.


In [23]:
piperun=generative_pipeline.run(
    query='What did the committe investigate?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)

The Senate Intelligence Committee investigated the CIA's detention and interrogation program. The Committee did not investigate whether the program was legal or not, but it did investigate whether it was ethical or not. It also investigated whether or not the program violated the Geneva Conventions.


In [24]:
piperun=generative_pipeline.run(
    query='Did the committee conclude that the CIA detention and interrogation program was ethical?', params={'Retriever': {'top_k': 5}} #output varies with different top_k values
)
for answer in piperun['answers']:
    print(answer.answer)

No, the committee did not conclude that the CIA detention and interrogation program was ethical.


In [25]:
piperun=generative_pipeline.run(
    query='What are black sites?', params={'Retriever': {'top_k': 5}}
)
piperun['answers'][0].answer

"A black site is a detention facility where a person is held indefinitely without trial. It's not a prison, it's a place where people are held without trial for a long period of time without being charged with a crime."